# Test of the clasification pipeline

In [1]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/ihranicky/git/domainradar-clf/classifiers


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [2]:
create_test_parquet = True

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    input_files = [
        {'file': 'testdata/misp_2402.parquet', 'max_rows': 500, 'label': 'phishing'},
        {'file': 'testdata/benign_2312.parquet', 'max_rows': 2000, 'label': 'benign'},
        {'file': 'testdata/malware_bp.parquet', 'max_rows': 500, 'label': 'malware'},
        {'file': 'testdata/dga_2310.parquet', 'max_rows': 500, 'label': 'dga'},
    ]

    # Number of rows to select in total
    n_rows = 1000
    
    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file'])
    
    # Limit the number of rows if necessary for the first file
    if len(combined_df) > first_file_info['max_rows']:
        combined_df = combined_df.sample(n=first_file_info['max_rows'], random_state=1)
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file'])
        
        # Limit the number of rows if necessary
        if len(df) > file_info['max_rows']:
            df = df.sample(n=file_info['max_rows'], random_state=1)
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = None
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Randomly select n_rows rows from the combined DataFrame
    selected_rows = combined_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility
    
    # Save the selected rows to a new Parquet file
    selected_rows.to_parquet('testdata/sample.parquet')

/tmp/ipykernel_74092/946921797.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([combined_df, df], ignore_index=True)


## Run classification

In [3]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/sample.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

In [4]:
import pandas as pd
import numpy as np
from pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

# Determine the number of chunks
num_chunks = (len(input_df) + CHUNK_SIZE - 1) // CHUNK_SIZE if CHUNK_SIZE > 0 else 1

# Process the dataframe in chunks
for i in range(num_chunks):
    if CHUNK_SIZE > 0:
        start_idx = i * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        chunk_df = input_df[start_idx:end_idx]
    else:
        chunk_df = input_df

    # Perform your classification or processing on the working_df here
    print(f"===== Processing chunk {i+1}/{num_chunks} =====")

    chunk_without_label = chunk_df.drop(columns=['label']) # Label should not be known to classifiers
    chunk_results = clf.classify_domains(chunk_without_label)

    for result in chunk_results:
        print(result)
     
    print(f"===== Chunk {i+1}/{num_chunks} completed. =====")

    break

2024-05-27 12:15:05.053041: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 12:15:05.053065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 12:15:05.053786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-27 12:15:05.058269: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 12:15:05.723937: W tensorflow/compiler/tf2

CNN model created
CNN model created
===== Processing chunk 1/34 =====


/home/ihranicky/git/domainradar-clf/classifiers/preprocessor.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(-1, inplace=True)
/home/ihranicky/git/domainradar-clf/classifiers/preprocessor.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(-1, inplace=True)
/home/ihranicky/git/domainradar-clf/classifiers/Clf_dga_binary_nn.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data.fillna(-1, 

                             domain_name  dns_available  dns_nonzero  \
1322                 cocinortemtb.com.do          1.000        0.725   
975                beautysalon.melbourne          0.600        0.125   
3085                          irivkte.me          0.475        0.000   
1263                          dtx.gov.az          1.000        0.700   
1533                      emmavoyages.fr          0.600        0.125   
2508        www.mail.additionaltones.org          0.600        0.200   
3228                 xyhtfmk2i46hibpv.ru          0.475        0.000   
3308      f90j6uci30xv1t10lsz1p2vfob.net          0.475        0.000   
3123                ysjsjaiqqkjdgasy.net          0.475        0.000   
330                         44445344.xyz          0.600        0.125   
2359                       mail.inbox.lv          0.600        0.200   
297                  conecct-ter.web.app          0.600        0.225   
2026                     cdn7.playep.pro          0.600        0

## Optional: Generate preliminary results for training the final aggregation classifier

In [18]:
import pandas as pd
import numpy as np
from pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

input_df = input_df.sample(frac=1).reset_index(drop=True)

preliminary_results_df = clf.generate_preliminary_results(input_df, output_file="test_preliminary_results.parquet")

CNN model created
CNN model created


/home/ihranicky/git/domainradar-clf/classifiers/preprocessor.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(-1, inplace=True)
/home/ihranicky/git/domainradar-clf/classifiers/preprocessor.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(-1, inplace=True)
/home/ihranicky/git/domainradar-clf/classifiers/Clf_dga_binary_nn.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data.fillna(-1, 

In [19]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
0,tbxvqnymryxhj.eu,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,1.000000,0.056259,1.000000,0.989823,0.999721
1,dwt.com.ro,1.000000,0.700000,1.000000,0.833333,1.000000,1.000000,0.958333,0.458333,1.000000,0.944444,benign,0.000000,0.001020,0.000000,0.000000,0.204216
2,assets.woolf.university,0.600000,0.225000,1.000000,0.458333,1.000000,0.625000,1.000000,0.708333,1.000000,0.722222,benign,0.000000,0.000250,0.000000,0.000015,0.000000
3,sqnsoauavvqnckkgn.me,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,1.000000,0.188214,1.000000,0.984668,0.999996
4,superhouse.mx,1.000000,0.750000,1.000000,0.458333,1.000000,0.625000,1.000000,0.583333,1.000000,0.722222,phishing,0.000000,0.788159,0.000000,0.002060,0.008999
5,miuportal.manarat.ac.bd,0.650000,0.275000,1.000000,0.833333,1.000000,1.000000,0.833333,0.291667,1.000000,1.000000,benign,0.000000,0.001030,0.000000,0.000016,0.000000
6,monteverde.immo,1.000000,0.750000,1.000000,0.458333,1.000000,0.875000,1.000000,0.875000,1.000000,0.944444,benign,1.000000,0.000036,1.000000,0.000000,0.000006
7,svgarage.rivne.ua,1.000000,0.725000,1.000000,0.458333,1.000000,0.875000,1.000000,0.708333,1.000000,1.000000,benign,0.000000,0.000310,0.000000,0.000000,0.000000
8,gwtnvjwtfwovduppg.pro,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,1.000000,0.313704,1.000000,0.996827,0.999990
9,0x.build,0.950000,0.625000,0.041667,0.000000,1.000000,0.625000,1.000000,0.916667,1.000000,1.000000,benign,1.000000,0.000070,1.000000,0.000001,0.093817


In [20]:
preliminary_results_df[["domain_name", "label", "phishing_cnn_result", "phishing_lgbm_result", "malware_cnn_result", "malware_xgboost_result", "dga_binary_nn_result"]]

,domain_name,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
0,tbxvqnymryxhj.eu,dga,1.000000,0.056259,1.000000,0.989823,0.999721
1,dwt.com.ro,benign,0.000000,0.001020,0.000000,0.000000,0.204216
2,assets.woolf.university,benign,0.000000,0.000250,0.000000,0.000015,0.000000
3,sqnsoauavvqnckkgn.me,dga,1.000000,0.188214,1.000000,0.984668,0.999996
4,superhouse.mx,phishing,0.000000,0.788159,0.000000,0.002060,0.008999
5,miuportal.manarat.ac.bd,benign,0.000000,0.001030,0.000000,0.000016,0.000000
6,monteverde.immo,benign,1.000000,0.000036,1.000000,0.000000,0.000006
7,svgarage.rivne.ua,benign,0.000000,0.000310,0.000000,0.000000,0.000000
8,gwtnvjwtfwovduppg.pro,dga,1.000000,0.313704,1.000000,0.996827,0.999990
9,0x.build,benign,1.000000,0.000070,1.000000,0.000001,0.093817
